In [461]:
import pandas as pd

In [462]:
df=pd.read_csv('/Users/Abdul/Desktop/MMA/Enterprise Data Science/Revenue-Radar/Data/train_dejsonified.csv', parse_dates=['date'])

/var/folders/rj/g3r71fsx2vnbbbjb9v368gr00000gp/T/ipykernel_4465/3898126540.py:1: DtypeWarning: Columns (2,54) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/Users/Abdul/Desktop/MMA/Enterprise Data Science/Revenue-Radar/Data/train_dejsonified.csv', parse_dates=['date'])


In [463]:
# Exploring the dataset; checking the data types of the columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 55 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   channelGrouping                      903653 non-null  object        
 1   date                                 903653 non-null  datetime64[ns]
 2   fullVisitorId                        903653 non-null  object        
 3   sessionId                            903653 non-null  object        
 4   socialEngagementType                 903653 non-null  object        
 5   visitId                              903653 non-null  int64         
 6   visitNumber                          903653 non-null  int64         
 7   visitStartTime                       903653 non-null  int64         
 8   continent                            903653 non-null  object        
 9   subContinent                         903653 non-null  object        
 

In [464]:
# Checking null values in the dataset
# Note that null values in the transactionRevenue column means that the customer did not make a purchase
df.isnull().sum()

channelGrouping                             0
date                                        0
fullVisitorId                               0
sessionId                                   0
socialEngagementType                        0
visitId                                     0
visitNumber                                 0
visitStartTime                              0
continent                                   0
subContinent                                0
country                                     0
region                                      0
metro                                       0
city                                        0
cityId                                      0
networkDomain                               0
latitude                                    0
longitude                                   0
networkLocation                             0
browser                                     0
browserVersion                              0
browserSize                       

#### Preparing the dataset for the classification model

In [465]:
df.columns.to_list()

['channelGrouping',
 'date',
 'fullVisitorId',
 'sessionId',
 'socialEngagementType',
 'visitId',
 'visitNumber',
 'visitStartTime',
 'continent',
 'subContinent',
 'country',
 'region',
 'metro',
 'city',
 'cityId',
 'networkDomain',
 'latitude',
 'longitude',
 'networkLocation',
 'browser',
 'browserVersion',
 'browserSize',
 'operatingSystem',
 'operatingSystemVersion',
 'isMobile',
 'mobileDeviceBranding',
 'mobileDeviceModel',
 'mobileInputSelector',
 'mobileDeviceInfo',
 'mobileDeviceMarketingName',
 'flashVersion',
 'language',
 'screenColors',
 'screenResolution',
 'deviceCategory',
 'visits',
 'hits',
 'pageviews',
 'bounces',
 'newVisits',
 'transactionRevenue',
 'campaign',
 'source',
 'medium',
 'keyword',
 'adwordsClickInfo.criteriaParameters',
 'isTrueDirect',
 'referralPath',
 'adwordsClickInfo.page',
 'adwordsClickInfo.slot',
 'adwordsClickInfo.gclId',
 'adwordsClickInfo.adNetworkType',
 'adwordsClickInfo.isVideoAd',
 'adContent',
 'campaignCode']

In [466]:
# Checking the number of unique values in each column
df.nunique()

channelGrouping                             8
date                                      366
fullVisitorId                          723803
sessionId                              902755
socialEngagementType                        1
visitId                                886303
visitNumber                               384
visitStartTime                         887159
continent                                   6
subContinent                               23
country                                   222
region                                    376
metro                                      94
city                                      649
cityId                                      1
networkDomain                           28064
latitude                                    1
longitude                                   1
networkLocation                             1
browser                                    54
browserVersion                              1
browserSize                       

In [467]:
# Checking the unique values in the 'bounces' column
df['bounces'].value_counts()

bounces
1.0    450630
Name: count, dtype: int64

In [468]:
# Replacing the null values in the 'bounces' column with 0
df['bounces'].fillna(0, inplace=True)
# Checking the unique values in the 'bounces' column after replacing the null values
df['bounces'].value_counts()

bounces
0.0    453023
1.0    450630
Name: count, dtype: int64

In [469]:
# Finding the columns with constant values
constant_columns = [col for col in df.columns if df[col].nunique() == 1]     
constant_columns

['socialEngagementType',
 'cityId',
 'latitude',
 'longitude',
 'networkLocation',
 'browserVersion',
 'browserSize',
 'operatingSystemVersion',
 'mobileDeviceBranding',
 'mobileDeviceModel',
 'mobileInputSelector',
 'mobileDeviceInfo',
 'mobileDeviceMarketingName',
 'flashVersion',
 'language',
 'screenColors',
 'screenResolution',
 'visits',
 'newVisits',
 'adwordsClickInfo.criteriaParameters',
 'isTrueDirect',
 'adwordsClickInfo.isVideoAd',
 'campaignCode']

### Dealing with Useless Variables

Upon inspecting the dataset to determine the useful columns for the use-case and identifying which columns will be dropped, the following observations were made:

- The following columns are IDs and are not useful for modeling but may be needed for eda. Will drop before modeling
  - `fullVisitorId`
  - `sessionId`
  - `visitId`
  - `cityId`

- The following columns are constant across all observations; therefore, all except `bounces` should be dropped. The `bounces` column contains null values and 1s, indicating that the observations with null values represent users who take actions before exiting:
  - `socialEngagementType`
  - `latitude`
  - `longitude`
  - `networkLocation`
  - `browserVersion`
  - `browserSize`
  - `operatingSystemVersion`
  - `mobileDeviceBranding`
  - `mobileDeviceModel`
  - `mobileInputSelector`
  - `mobileDeviceInfo`
  - `mobileDeviceMarketingName`
  - `flashVersion`
  - `language`
  - `screenColors`
  - `screenResolution`
  - `visits`
  - `newVisits`
  - `adwordsClickInfo.criteriaParameters`
  - `isTrueDirect`
  - `adwordsClickInfo.isVideoAd`
  - `campaignCode`
- The `networkDomain` column has too many unique values; therefore, it should be dropped

In [470]:

# dropping columns with many unique values
df.drop(['networkDomain','visitStartTime','visitNumber'], axis=1, inplace=True)

In [471]:
# Checking the new dimensions of the dataset
df.shape

(903653, 52)

In [472]:

# dropping the columns with constant values
df.drop(constant_columns, axis=1, inplace=True)

In [473]:
# checking the shape of the dataset after dropping the columns with constant values 
df.shape

(903653, 29)

In [474]:
# Calculaing the proportion of missing values in each column
missing_values = df.isnull().mean().sort_values(ascending=False)
missing_values

adContent                         0.987887
transactionRevenue                0.987257
adwordsClickInfo.adNetworkType    0.976252
adwordsClickInfo.slot             0.976252
adwordsClickInfo.page             0.976252
adwordsClickInfo.gclId            0.976140
referralPath                      0.633774
keyword                           0.556551
pageviews                         0.000111
hits                              0.000000
medium                            0.000000
source                            0.000000
campaign                          0.000000
bounces                           0.000000
channelGrouping                   0.000000
date                              0.000000
isMobile                          0.000000
operatingSystem                   0.000000
browser                           0.000000
city                              0.000000
metro                             0.000000
region                            0.000000
country                           0.000000
subContinen

In [475]:
# Checking the number of null values in the dataset
df.isnull().sum()

channelGrouping                        0
date                                   0
fullVisitorId                          0
sessionId                              0
visitId                                0
continent                              0
subContinent                           0
country                                0
region                                 0
metro                                  0
city                                   0
browser                                0
operatingSystem                        0
isMobile                               0
deviceCategory                         0
hits                                   0
pageviews                            100
bounces                                0
transactionRevenue                892138
campaign                               0
source                                 0
medium                                 0
keyword                           502929
referralPath                      572712
adwordsClickInfo

In [476]:
df.head()

,channelGrouping,date,fullVisitorId,sessionId,visitId,continent,subContinent,country,region,metro,...,campaign,source,medium,keyword,referralPath,adwordsClickInfo.page,adwordsClickInfo.slot,adwordsClickInfo.gclId,adwordsClickInfo.adNetworkType,adContent
0,Organic Search,2016-09-02,1131660440785968503,1131660440785968503_1472830385,1472830385,Asia,Western Asia,Turkey,Izmir,(not set),...,(not set),google,organic,(not provided),NaN,NaN,NaN,NaN,NaN,NaN
1,Organic Search,2016-09-02,377306020877927890,377306020877927890_1472880147,1472880147,Oceania,Australasia,Australia,not available in demo dataset,not available in demo dataset,...,(not set),google,organic,(not provided),NaN,NaN,NaN,NaN,NaN,NaN
2,Organic Search,2016-09-02,3895546263509774583,3895546263509774583_1472865386,1472865386,Europe,Southern Europe,Spain,Community of Madrid,(not set),...,(not set),google,organic,(not provided),NaN,NaN,NaN,NaN,NaN,NaN
3,Organic Search,2016-09-02,4763447161404445595,4763447161404445595_1472881213,1472881213,Asia,Southeast Asia,Indonesia,not available in demo dataset,not available in demo dataset,...,(not set),google,organic,google + online,NaN,NaN,NaN,NaN,NaN,NaN
4,Organic Search,2016-09-02,27294437909732085,27294437909732085_1472822600,1472822600,Europe,Northern Europe,United Kingdom,not available in demo dataset,not available in demo dataset,...,(not set),google,organic,(not provided),NaN,NaN,NaN,NaN,NaN,NaN


In [477]:
df.pageviews.fillna(0, inplace=True)

In [478]:
df.isna().sum()

channelGrouping                        0
date                                   0
fullVisitorId                          0
sessionId                              0
visitId                                0
continent                              0
subContinent                           0
country                                0
region                                 0
metro                                  0
city                                   0
browser                                0
operatingSystem                        0
isMobile                               0
deviceCategory                         0
hits                                   0
pageviews                              0
bounces                                0
transactionRevenue                892138
campaign                               0
source                                 0
medium                                 0
keyword                           502929
referralPath                      572712
adwordsClickInfo

In [479]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 29 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   channelGrouping                 903653 non-null  object        
 1   date                            903653 non-null  datetime64[ns]
 2   fullVisitorId                   903653 non-null  object        
 3   sessionId                       903653 non-null  object        
 4   visitId                         903653 non-null  int64         
 5   continent                       903653 non-null  object        
 6   subContinent                    903653 non-null  object        
 7   country                         903653 non-null  object        
 8   region                          903653 non-null  object        
 9   metro                           903653 non-null  object        
 10  city                            903653 non-null  object 

In [480]:
# checking the number of unique visitor ids
df.fullVisitorId.nunique()

723803

In [482]:
df['transactionRevenue'].fillna(0, inplace=True)

In [483]:
df['Conversion'] = df['transactionRevenue'].apply(lambda x: 1 if x > 0 else 0)

In [484]:
# checking the number of conversions for each user
coversion_by_user=dict(df.groupby('fullVisitorId')['Conversion'].sum().sort_values(ascending=False))
coversion_by_user

{7813149961404844386: 31,
 1957458976293878100: 16,
 4984366501121503466: 16,
 6760732402251466726: 13,
 608915197735218105: 12,
 2402527199731150932: 11,
 7311242886083854158: 11,
 9089132392240687728: 10,
 8197879643797712877: 10,
 9029794295932939024: 9,
 9377429831454005466: 9,
 4835082938415020542: 8,
 7463172420271311409: 8,
 8436426603099391262: 8,
 1956307607572137989: 7,
 1814166460229302850: 7,
 372706149688864468: 7,
 8657427332734176422: 6,
 1639589047263376521: 6,
 8851822767477687842: 6,
 2074164338647079047: 6,
 4808974885561715322: 6,
 1285137808569536764: 6,
 2411322974724385937: 6,
 9807072102349488174: 6,
 3287424336851494398: 6,
 1063651652107093288: 6,
 3857043812510146001: 6,
 6010250598436085923: 6,
 1759779356678951325: 6,
 777922178356486144: 6,
 4358918846442674981: 5,
 119870259714905967: 5,
 9372618001980854712: 5,
 220695720492664721: 5,
 6476783278979674523: 5,
 4921413806940784676: 5,
 6006657047016577071: 5,
 9894955795481014038: 5,
 4578640586284138624:

In [485]:
df['at_least_one_conversion']=df.groupby('fullVisitorId')['Conversion'].transform('sum').apply(lambda x: 1 if x>0 else 0)

In [486]:
df.channelGrouping.value_counts()

channelGrouping
Organic Search    381561
Social            226117
Direct            143026
Referral          104838
Paid Search        25326
Affiliates         16403
Display             6262
(Other)              120
Name: count, dtype: int64

In [487]:
# checking the number of conversions for each user
conversions=pd.DataFrame(coversion_by_user.items(), columns=['fullVisitorId', 'Conversion'])
conversions

,fullVisitorId,Conversion
0,7813149961404844386,31
1,1957458976293878100,16
2,4984366501121503466,16
3,6760732402251466726,13
4,608915197735218105,12
...,...,...
723798,2790806198515368990,0
723799,2790839817455884413,0
723800,2790841192617951644,0
723801,2790844361171199630,0


In [488]:
# Ensuring the DataFrame is sorted by fullVisitorId and date
df.sort_values(by=['fullVisitorId', 'date'], inplace=True)

In [489]:
# Grouping by fullVisitorId and get the first and last visit's channelGrouping
first_visit_channel = df.groupby('fullVisitorId')['channelGrouping'].first().reset_index(name='FirstChannelVisit')
last_visit_channel = df.groupby('fullVisitorId')['channelGrouping'].last().reset_index(name='LastChannelVisit')

In [490]:
# Merge the first and last visit channel dataframes
channels_first_last = pd.merge(first_visit_channel, last_visit_channel, on='fullVisitorId', how='inner')
channels_first_last.sample(40) ## other columns will be merged to this dataframe
channels_first_last.shape

(723803, 3)

In [491]:
df_unique = df.drop_duplicates(subset='fullVisitorId', keep='first')
df_unique.shape

(723803, 31)

In [492]:
channels_first_last = pd.merge(channels_first_last, df_unique[['fullVisitorId', 'at_least_one_conversion', 'country','continent','subContinent']].drop_duplicates(), on='fullVisitorId', how='inner')
channels_first_last.shape

(723803, 7)

In [493]:
# checking which users had different first and last channel visits
channels_first_last[channels_first_last['FirstChannelVisit'] != channels_first_last['LastChannelVisit']]

,fullVisitorId,FirstChannelVisit,LastChannelVisit,at_least_one_conversion,country,continent,subContinent
120,357767462604476,Direct,Display,0,United States,Americas,Northern America
121,358027242270102,Paid Search,Organic Search,0,United States,Americas,Northern America
184,562678147042735,Organic Search,Referral,1,United States,Americas,Northern America
219,650183049109374,Organic Search,Paid Search,0,India,Asia,Southern Asia
229,677695778949032,Direct,Referral,0,United States,Americas,Northern America
...,...,...,...,...,...,...,...
723415,9913166595926807960,Organic Search,Referral,0,United States,Americas,Northern America
723519,9937314053887872916,Affiliates,Organic Search,0,Canada,Americas,Northern America
723581,9952660341541977540,Organic Search,Referral,0,United States,Americas,Northern America
723620,9960561123114745936,Organic Search,Referral,0,Spain,Europe,Southern Europe


### Most users had the same first and last channel visits except for 17296 users who had different first and last channel visits

In [494]:
# creating another variable for total number of visits for each user
total_visits_by_user=dict(df['fullVisitorId'].value_counts().reset_index(name='TotalVisits').values)
total_visits_by_user

{1957458976293878100: 262,
 824839726118485274: 245,
 1856749147915772585: 188,
 3608475193341679870: 186,
 3269834865385146569: 152,
 7634897085866546110: 145,
 720311197761340948: 141,
 232377434237234751: 134,
 4038076683036146727: 132,
 3694234028523165868: 122,
 949718915643445721: 119,
 3525537916960843419: 115,
 3148617623907142276: 103,
 6018775317735347795: 103,
 6254908847172458133: 101,
 2194592743396253647: 100,
 9801276214964695322: 100,
 9609104828919391966: 98,
 7498695963354635199: 98,
 3937673380007666721: 90,
 2082625651279391786: 82,
 7445235885559107095: 80,
 7813149961404844386: 80,
 4913801338365738862: 77,
 4578640586284138624: 72,
 9534537897118577344: 70,
 3041133261614133977: 70,
 678102787097322550: 70,
 6588500311054802771: 68,
 4731929753431036485: 67,
 9292327473106748702: 65,
 3404435547562826616: 64,
 9320456289108916892: 64,
 834799491548640007: 64,
 4211345503073247187: 63,
 5208937953046059083: 63,
 5167931477427215448: 63,
 9846596419685375593: 63,
 

In [495]:
# Creating a variable to capture number of visits through each channel for each user
channel_visits_by_user=dict(df.groupby('fullVisitorId')['channelGrouping'].value_counts())
channel_visits_by_user

{(4823595352351, 'Organic Search'): 1,
 (5103959234087, 'Organic Search'): 1,
 (10278554503158, 'Organic Search'): 1,
 (20424342248747, 'Organic Search'): 1,
 (26722803385797, 'Organic Search'): 1,
 (27376579751715, 'Organic Search'): 1,
 (33471059618621, 'Social'): 1,
 (35794135966385, 'Direct'): 1,
 (39460501403861, 'Social'): 1,
 (40862739425590, 'Paid Search'): 2,
 (45417921646651, 'Referral'): 1,
 (47810105303746, 'Direct'): 1,
 (48421062322244, 'Direct'): 1,
 (52246474382344, 'Social'): 1,
 (58970809397690, 'Organic Search'): 1,
 (61609983499251, 'Social'): 1,
 (62267706107999, 'Affiliates'): 1,
 (62349695125717, 'Referral'): 1,
 (63834990385751, 'Social'): 1,
 (64767209884626, 'Organic Search'): 2,
 (68403966359845, 'Organic Search'): 1,
 (72202462202136, 'Organic Search'): 1,
 (75746323734481, 'Organic Search'): 1,
 (77980744724824, 'Organic Search'): 1,
 (82001414223176, 'Organic Search'): 1,
 (85059828173212, 'Direct'): 1,
 (90085033332104, 'Organic Search'): 1,
 (90134415725

In [496]:
# converting the dictionary to a list of dictionaries
list_of_dicts = []
for (visitor_id, channel), visits in channel_visits_by_user.items():
    list_of_dicts.append({'fullVisitorId': visitor_id, channel: visits})

# Creating a DataFrame
df_channel_visits = pd.DataFrame(list_of_dicts)

# Combining all dictionaries representing the same fullVisitorId
df_channel_visits = df_channel_visits.groupby('fullVisitorId', as_index=False).sum()

df_channel_visits

,fullVisitorId,Organic Search,Social,Direct,Paid Search,Referral,Affiliates,Display,(Other)
0,4823595352351,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5103959234087,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10278554503158,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20424342248747,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,26722803385797,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
723798,9999470222842102470,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
723799,9999493741609373843,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
723800,9999607385222235728,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
723801,9999761280717362235,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [497]:
# creating another variable for total number of pageviews for each user 
total_pageviews_by_user=dict(df.groupby('fullVisitorId')['pageviews'].sum().reset_index(name='TotalPageviews').values)
total_pageviews_by_user

{4823595352351: 1.0,
 5103959234087: 8.0,
 10278554503158: 8.0,
 20424342248747: 13.0,
 26722803385797: 2.0,
 27376579751715: 5.0,
 33471059618621: 1.0,
 35794135966385: 1.0,
 39460501403861: 2.0,
 40862739425590: 5.0,
 45417921646651: 2.0,
 47810105303746: 1.0,
 48421062322244: 1.0,
 52246474382344: 1.0,
 58970809397690: 5.0,
 61609983499251: 2.0,
 62267706107999: 1.0,
 62349695125717: 1.0,
 63834990385751: 2.0,
 64767209884626: 7.0,
 68403966359845: 2.0,
 72202462202136: 1.0,
 75746323734481: 3.0,
 77980744724824: 4.0,
 82001414223176: 12.0,
 85059828173212: 1.0,
 90085033332104: 1.0,
 90134415725564: 1.0,
 93957001069502: 2.0,
 95855025602688: 1.0,
 97118899221776: 2.0,
 101322961649621: 1.0,
 109798218040435: 4.0,
 113091074918758: 6.0,
 114156543135683: 1.0,
 120513263852408: 23.0,
 123950480336950: 1.0,
 132848834502720: 6.0,
 139156957304532: 3.0,
 142967912891615: 1.0,
 143747325766784: 1.0,
 144663615238931: 1.0,
 150357983459565: 1.0,
 152474579038632: 2.0,
 152560646457113: 

In [498]:
# creating a variable for total number of bounces for each user
total_bounces_by_user=dict(df.groupby('fullVisitorId')['bounces'].sum().reset_index(name='TotalBounces').values)
total_bounces_by_user

{4823595352351: 1.0,
 5103959234087: 0.0,
 10278554503158: 0.0,
 20424342248747: 0.0,
 26722803385797: 0.0,
 27376579751715: 0.0,
 33471059618621: 1.0,
 35794135966385: 1.0,
 39460501403861: 0.0,
 40862739425590: 0.0,
 45417921646651: 0.0,
 47810105303746: 1.0,
 48421062322244: 1.0,
 52246474382344: 1.0,
 58970809397690: 0.0,
 61609983499251: 0.0,
 62267706107999: 1.0,
 62349695125717: 1.0,
 63834990385751: 0.0,
 64767209884626: 1.0,
 68403966359845: 0.0,
 72202462202136: 1.0,
 75746323734481: 0.0,
 77980744724824: 0.0,
 82001414223176: 0.0,
 85059828173212: 1.0,
 90085033332104: 1.0,
 90134415725564: 1.0,
 93957001069502: 0.0,
 95855025602688: 1.0,
 97118899221776: 0.0,
 101322961649621: 1.0,
 109798218040435: 1.0,
 113091074918758: 0.0,
 114156543135683: 1.0,
 120513263852408: 0.0,
 123950480336950: 1.0,
 132848834502720: 0.0,
 139156957304532: 0.0,
 142967912891615: 1.0,
 143747325766784: 1.0,
 144663615238931: 1.0,
 150357983459565: 1.0,
 152474579038632: 0.0,
 152560646457113: 0.0

In [499]:
# Creating another variable for number of visters per device category
visits_by_device=dict(df.groupby('fullVisitorId')['deviceCategory'].value_counts())
visits_by_device

{(4823595352351, 'desktop'): 1,
 (5103959234087, 'mobile'): 1,
 (10278554503158, 'desktop'): 1,
 (20424342248747, 'desktop'): 1,
 (26722803385797, 'desktop'): 1,
 (27376579751715, 'desktop'): 1,
 (33471059618621, 'desktop'): 1,
 (35794135966385, 'mobile'): 1,
 (39460501403861, 'desktop'): 1,
 (40862739425590, 'desktop'): 2,
 (45417921646651, 'desktop'): 1,
 (47810105303746, 'desktop'): 1,
 (48421062322244, 'desktop'): 1,
 (52246474382344, 'desktop'): 1,
 (58970809397690, 'mobile'): 1,
 (61609983499251, 'desktop'): 1,
 (62267706107999, 'desktop'): 1,
 (62349695125717, 'desktop'): 1,
 (63834990385751, 'desktop'): 1,
 (64767209884626, 'mobile'): 2,
 (68403966359845, 'tablet'): 1,
 (72202462202136, 'mobile'): 1,
 (75746323734481, 'desktop'): 1,
 (77980744724824, 'desktop'): 1,
 (82001414223176, 'desktop'): 1,
 (85059828173212, 'mobile'): 1,
 (90085033332104, 'mobile'): 1,
 (90134415725564, 'mobile'): 1,
 (93957001069502, 'desktop'): 1,
 (95855025602688, 'desktop'): 1,
 (97118899221776, 'de

In [500]:
# converting the dictionary to a list of dictionaries
list_of_dicts = []
for (visitor_id, device), visits in visits_by_device.items():
    list_of_dicts.append({'fullVisitorId': visitor_id, device: visits})

# Creating a DataFrame
df_visits_by_device = pd.DataFrame(list_of_dicts)

# Combining all dictionaries representing the same fullVisitorId
df_visits_by_device = df_visits_by_device.groupby('fullVisitorId', as_index=False).sum()

df_visits_by_device

,fullVisitorId,desktop,mobile,tablet
0,4823595352351,1.0,0.0,0.0
1,5103959234087,0.0,1.0,0.0
2,10278554503158,1.0,0.0,0.0
3,20424342248747,1.0,0.0,0.0
4,26722803385797,1.0,0.0,0.0
...,...,...,...,...
723798,9999470222842102470,1.0,0.0,0.0
723799,9999493741609373843,1.0,0.0,0.0
723800,9999607385222235728,1.0,0.0,0.0
723801,9999761280717362235,0.0,0.0,1.0


In [501]:
# Creating another variable for the first and last session pageviews for each user   
first_session_pageviews=dict(df.groupby('fullVisitorId')['pageviews'].first().reset_index(name='FirstSessionPageviews').values)
last_session_pageviews=dict(df.groupby('fullVisitorId')['pageviews'].last().reset_index(name='LastSessionPageviews').values)

In [502]:
df['campaign'].value_counts()   

campaign
(not set)                                          865347
Data Share Promo                                    16403
AW - Dynamic Search Ads Whole Site                  14244
AW - Accessories                                     7070
test-liyuhz                                           392
AW - Electronics                                       96
Retail (DO NOT EDIT owners nophakun and tianyu)        50
AW - Apparel                                           46
All Products                                            4
Data Share                                              1
Name: count, dtype: int64

In [503]:
df['campaign']=df['campaign'].apply(lambda x: 0 if x=='(not set)' else 1)

In [504]:
df['campaign'].value_counts()   

campaign
0    865347
1     38306
Name: count, dtype: int64

In [505]:
# creating a variable for number of campaign visits for each user
campaign_visits_by_user=dict(df.groupby('fullVisitorId')['campaign'].sum().reset_index(name='CampaignVisits').values)
campaign_visits_by_user

{4823595352351: 0,
 5103959234087: 0,
 10278554503158: 0,
 20424342248747: 0,
 26722803385797: 0,
 27376579751715: 0,
 33471059618621: 0,
 35794135966385: 0,
 39460501403861: 0,
 40862739425590: 2,
 45417921646651: 0,
 47810105303746: 0,
 48421062322244: 0,
 52246474382344: 0,
 58970809397690: 0,
 61609983499251: 0,
 62267706107999: 1,
 62349695125717: 0,
 63834990385751: 0,
 64767209884626: 0,
 68403966359845: 0,
 72202462202136: 0,
 75746323734481: 0,
 77980744724824: 0,
 82001414223176: 0,
 85059828173212: 0,
 90085033332104: 0,
 90134415725564: 0,
 93957001069502: 0,
 95855025602688: 0,
 97118899221776: 0,
 101322961649621: 0,
 109798218040435: 0,
 113091074918758: 0,
 114156543135683: 0,
 120513263852408: 0,
 123950480336950: 0,
 132848834502720: 0,
 139156957304532: 0,
 142967912891615: 0,
 143747325766784: 0,
 144663615238931: 0,
 150357983459565: 0,
 152474579038632: 0,
 152560646457113: 1,
 158916953288213: 0,
 159853817888337: 0,
 166465265517759: 0,
 166529102473253: 0,
 167

In [521]:

# Converting dictionaries to DataFrames
df_total_bounces = pd.DataFrame(list(total_bounces_by_user.items()), columns=['fullVisitorId', 'TotalBounces'])
df_total_visits = pd.DataFrame(list(total_visits_by_user.items()), columns=['fullVisitorId', 'TotalVisits'])
df_total_pageviews = pd.DataFrame(list(total_pageviews_by_user.items()), columns=['fullVisitorId', 'TotalPageviews'])
df_first_session_pageviews = pd.DataFrame(list(first_session_pageviews.items()), columns=['fullVisitorId', 'FirstSessionPageviews'])
df_last_session_pageviews = pd.DataFrame(list(last_session_pageviews.items()), columns=['fullVisitorId', 'LastSessionPageviews'])
df_campaign_visits = pd.DataFrame(list(campaign_visits_by_user.items()), columns=['fullVisitorId', 'CampaignVisits'])

# Start with the channels_first_last DataFrame
final_df = channels_first_last

#merging each DataFrame on 'fullVisitorId'
for dfr in [df_total_visits, df_total_bounces, df_channel_visits, df_total_pageviews, df_visits_by_device, df_first_session_pageviews, df_last_session_pageviews, conversions, df_campaign_visits]:
    final_df = pd.merge(final_df, dfr, on='fullVisitorId', how='inner')


final_df.head()

,fullVisitorId,FirstChannelVisit,LastChannelVisit,at_least_one_conversion,country,continent,subContinent,TotalVisits,TotalBounces,Organic Search,...,Display,(Other),TotalPageviews,desktop,mobile,tablet,FirstSessionPageviews,LastSessionPageviews,Conversion,CampaignVisits
0,4823595352351,Organic Search,Organic Search,0,China,Asia,Eastern Asia,1,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0,0
1,5103959234087,Organic Search,Organic Search,0,United States,Americas,Northern America,1,0.0,1.0,...,0.0,0.0,8.0,0.0,1.0,0.0,8.0,8.0,0,0
2,10278554503158,Organic Search,Organic Search,0,New Zealand,Oceania,Australasia,1,0.0,1.0,...,0.0,0.0,8.0,1.0,0.0,0.0,8.0,8.0,0,0
3,20424342248747,Organic Search,Organic Search,0,Peru,Americas,South America,1,0.0,1.0,...,0.0,0.0,13.0,1.0,0.0,0.0,13.0,13.0,0,0
4,26722803385797,Organic Search,Organic Search,0,United Kingdom,Europe,Northern Europe,1,0.0,1.0,...,0.0,0.0,2.0,1.0,0.0,0.0,2.0,2.0,0,0


In [524]:
final_df.shape

(723803, 25)

In [523]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 723803 entries, 0 to 723802
Data columns (total 25 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   fullVisitorId            723803 non-null  object 
 1   FirstChannelVisit        723803 non-null  object 
 2   LastChannelVisit         723803 non-null  object 
 3   at_least_one_conversion  723803 non-null  int64  
 4   country                  723803 non-null  object 
 5   continent                723803 non-null  object 
 6   subContinent             723803 non-null  object 
 7   TotalVisits              723803 non-null  int64  
 8   TotalBounces             723803 non-null  float64
 9   Organic Search           723803 non-null  float64
 10  Social                   723803 non-null  float64
 11  Direct                   723803 non-null  float64
 12  Paid Search              723803 non-null  float64
 13  Referral                 723803 non-null  float64
 14  Affi

In [525]:
for col in final_df.select_dtypes(include='float').columns:
    final_df[col] = final_df[col].astype('int')
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 723803 entries, 0 to 723802
Data columns (total 25 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   fullVisitorId            723803 non-null  object
 1   FirstChannelVisit        723803 non-null  object
 2   LastChannelVisit         723803 non-null  object
 3   at_least_one_conversion  723803 non-null  int64 
 4   country                  723803 non-null  object
 5   continent                723803 non-null  object
 6   subContinent             723803 non-null  object
 7   TotalVisits              723803 non-null  int64 
 8   TotalBounces             723803 non-null  int64 
 9   Organic Search           723803 non-null  int64 
 10  Social                   723803 non-null  int64 
 11  Direct                   723803 non-null  int64 
 12  Paid Search              723803 non-null  int64 
 13  Referral                 723803 non-null  int64 
 14  Affiliates          

In [526]:
final_df.head()

,fullVisitorId,FirstChannelVisit,LastChannelVisit,at_least_one_conversion,country,continent,subContinent,TotalVisits,TotalBounces,Organic Search,...,Display,(Other),TotalPageviews,desktop,mobile,tablet,FirstSessionPageviews,LastSessionPageviews,Conversion,CampaignVisits
0,4823595352351,Organic Search,Organic Search,0,China,Asia,Eastern Asia,1,1,1,...,0,0,1,1,0,0,1,1,0,0
1,5103959234087,Organic Search,Organic Search,0,United States,Americas,Northern America,1,0,1,...,0,0,8,0,1,0,8,8,0,0
2,10278554503158,Organic Search,Organic Search,0,New Zealand,Oceania,Australasia,1,0,1,...,0,0,8,1,0,0,8,8,0,0
3,20424342248747,Organic Search,Organic Search,0,Peru,Americas,South America,1,0,1,...,0,0,13,1,0,0,13,13,0,0
4,26722803385797,Organic Search,Organic Search,0,United Kingdom,Europe,Northern Europe,1,0,1,...,0,0,2,1,0,0,2,2,0,0


In [527]:
#saving the final DataFrame to a csv file
final_df.to_csv('/Users/Abdul/Desktop/MMA/Enterprise Data Science/final_df.csv', index=False)

In [518]:
final_df.at_least_one_conversion.value_counts()

at_least_one_conversion
0    713689
1     10114
Name: count, dtype: int64

In [520]:
final_df.columns.to_list()

['fullVisitorId',
 'FirstChannelVisit',
 'LastChannelVisit',
 'at_least_one_conversion',
 'country',
 'continent',
 'subContinent',
 'TotalVisits',
 'Organic Search',
 'Social',
 'Direct',
 'Paid Search',
 'Referral',
 'Affiliates',
 'Display',
 '(Other)',
 'TotalPageviews',
 'desktop',
 'mobile',
 'tablet',
 'FirstSessionPageviews',
 'LastSessionPageviews',
 'Conversion',
 'CampaignVisits']